In [12]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, \
    SeparableConv2D, GlobalAveragePooling2D
from keras.utils import to_categorical, Sequence
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import initializers
from numpy.random import seed
import numpy as np
from keras import applications
import json
import random
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
import imgaug.augmenters as iaa
import imgaug as ia
import tensorflow as tf    

In [24]:
tf.Session(config=tf.ConfigProto(log_device_placement=True)) 

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [3]:
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [4]:
labels = "./dataset/labels_for_test.json"
with open(labels) as label:
    data = json.load(label) 

In [5]:
def data_prepare(labels):
    data = {}
    for key in labels.keys():
        if labels[key] not in data:
            data.update({labels[key]: []})
        data[labels[key]].append(key)
    return data
data  = data_prepare(data)

In [6]:
max_len = 0
train_data = {}
val_data = {}
for i in data:
    if len(data[i]) > max_len:
        max_len = len(data[i])
    train_data.update({i: data[i][:int(0.9*len(data[i]))]})
    val_data.update({i: data[i][int(0.9*len(data[i])):]})

In [7]:
import pprint

In [8]:
img_width, img_height = 256, 256
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
    iaa.Fliplr(0.8),
    sometimes(iaa.Affine(
    rotate=(-18, 18),
)),
])


for label in train_data:
    if not os.path.exists('dataset/learning_data/' + label):
        os.makedirs('dataset/learning_data/' + label)
        for i, img_name in enumerate(train_data[label]):
            image = cv2.imread('dataset/data_for_test/' +  img_name)
            gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
            cv2.imwrite('dataset/learning_data/' + label + "/" + str(i) +".jpg", gray)
        for i in range(max_len-len(train_data[label])):
            image = cv2.imread('dataset/data_for_test/' + random.choice(train_data[label]))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            images_aug = seq(images=gray)
            cv2.imwrite('dataset/learning_data/' + label + "/" + str(len(train_data[label]) + i) +".jpg", images_aug)
    else:
        print("Folder with train data {} prepared".format(label))

for label in val_data:
    if not os.path.exists('dataset/validation_data/' + label):
        os.makedirs('dataset/validation_data/' + label)
        for i, img_name in enumerate(val_data[label]):
            image = cv2.imread('dataset/data_for_test/' +  img_name)
            gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
            cv2.imwrite('dataset/validation_data/' + label + "/" + str(i) +".jpg", gray)
    else:
        print("Folder with validation data {} prepared".format(label))

        
del(val_data)
del(train_data)
del(data)

Folder with train data stubble prepared
Folder with train data goatee_with_moustache prepared
Folder with train data shaven prepared
Folder with train data moustache prepared
Folder with train data classic_short prepared
Folder with train data classic_long prepared
Folder with train data goatee prepared
Folder with train data chin_curtain prepared
Folder with validation data stubble prepared
Folder with validation data goatee_with_moustache prepared
Folder with validation data shaven prepared
Folder with validation data moustache prepared
Folder with validation data classic_short prepared
Folder with validation data classic_long prepared
Folder with validation data goatee prepared
Folder with validation data chin_curtain prepared


In [3]:
img_width, img_height = 256, 256

In [4]:
train_data_dir = "dataset/learning_data/"
validation_data_dir = "dataset/validation_data/"
nb_train_samples = 400
nb_validation_samples = 2
epochs = 50
batch_size = 8
base_model = applications.VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu',
          kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None))(x)

predictions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# def build_model():
#     input_img = Input(shape=(img_width,img_height,3), name='ImageInput')
#     x = Conv2D(512, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
#     x = Conv2D(512, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
#     x = MaxPooling2D((2,2), name='pool1')(x)
    
#     x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
#     x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
#     x = MaxPooling2D((2,2), name='pool2')(x)
    
#     x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
#     x = BatchNormalization(name='bn1')(x)
#     x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
#     x = BatchNormalization(name='bn2')(x)
#     x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
#     x = MaxPooling2D((2,2), name='pool3')(x)
    
#     x = SeparableConv2D(64, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
#     x = BatchNormalization(name='bn3')(x)
#     x = SeparableConv2D(64, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
#     x = BatchNormalization(name='bn4')(x)
#     x = SeparableConv2D(64, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
#     x = MaxPooling2D((2,2), name='pool4')(x)
    
#     x = Flatten(name='flatten')(x)
#     x = Dense(64, activation='relu', name='fc1')(x)
#     x = Dropout(0.1, name='dropout2')(x)
#     x = Dense(8, activation='softmax', name='fc3')(x)
    
#     model = Model(inputs=input_img, outputs=x)
#     return model
# model =  build_model()


# for layer in base_model.layers:
#     layer.trainable = False

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

Found 2408 images belonging to 8 classes.
Found 205 images belonging to 8 classes.
Epoch 1/50
400/400 [==============================] - 36s 91ms/step - loss: 2.0154 - acc: 0.2125 - val_loss: 2.1437 - val_acc: 0.1875
Epoch 2/50
400/400 [==============================] - 35s 89ms/step - loss: 1.8263 - acc: 0.2994 - val_loss: 2.0386 - val_acc: 0.1250
Epoch 3/50
400/400 [==============================] - 36s 90ms/step - loss: 1.7158 - acc: 0.3606 - val_loss: 2.2071 - val_acc: 0.1250
Epoch 4/50
400/400 [==============================] - 36s 90ms/step - loss: 1.6215 - acc: 0.3878 - val_loss: 2.0895 - val_acc: 0.2500
Epoch 5/50
400/400 [==============================] - 36s 90ms/step - loss: 1.5508 - acc: 0.4119 - val_loss: 1.9152 - val_acc: 0.3750
Epoch 6/50
400/400 [==============================] - 37s 92ms/step - loss: 1.5012 - acc: 0.4372 - val_loss: 2.7304 - val_acc: 0.0625
Epoch 7/50
400/400 [==============================] - 37s 92ms/step - loss: 1.4419 - acc: 0.4713 - val_loss: 2.17

In [8]:
img = glob("dataset/validation_data/chin_curtain/*")

In [48]:
img = np.array([cv2.imread("dataset/validation_data/goatee/1.jpg")])

In [49]:
model.predict(img)

array([[0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [15]:
model.predict(img)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.3176474e-19,
        0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [50]:
model.save('beard.h5')

In [9]:
classic_long = 4
goatee = 5
classic_short = 7

['dataset/validation_data/chin_curtain/']

In [24]:
class Seq(Sequence):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]["image"]
        batch_y = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]["label"]

        return (np.array([cv2.resize(cv2.imread(file_name, 1), (256, 256)) for file_name in batch_x]),
                keras.utils.to_categorical(np.array(batch_y), num_classes =8))

def build_model():
    input_img = Input(shape=(256,256,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.1, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.1, name='dropout2')(x)
    x = Dense(8, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model
model =  build_model()


opt = Adam(lr=0.0001, decay=1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

batch_size = 8
nb_epochs = 20

train_data_gen = Seq(data=data, batch_size=batch_size)

nb_train_steps = 30

history = model.fit_generator(generator=train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                               validation_data=(valid_data, valid_labels),callbacks=[es, chkpt])

NameError: name 'valid_data' is not defined